# LulaSafe API

This tutorial will guide you step by step on how to use the LulaSafe GraphQL API.

In [ ]:
import os
import json
from pprint import pp

def run_command(cmd):
    print(os.popen(cmd).read())

In [ ]:
run_command('pip install gql[all]')
run_command('pip install requests')

## Authentication
> **Warning**
>
> Until we add support for OpenID Connect client credentials flow, we need to perform some custom token retrieving actions

### 1. Read your credentials
> **Important**
>
> Create [`appsecrets.json`](../appsecrets.json) in the repo root and set your credentials into it
>
> ``` JSON
> {
>     "ClientId": "< Your Lula login >",
>     "ClientSecret": "< Your Lula password >"
> }
> 

In [ ]:
class LulaSafeOptions():
    def __init__(self, client_id, client_secret, base_url):
        self.client_id = client_id
        self.client_secret = client_secret
        self.base_url = base_url

    def __str__(self):
        return f"base_url: {self.base_url}{os.linesep}client_id{self.client_id}"
        
    @staticmethod
    def read_from_config(appsecrets_path, appsettings_path):
        secrets_object = None
        with open(appsecrets_path) as app_secrets:
            secrets_object = json.load(app_secrets)
        
        settings_object = None
        with open(appsettings_path) as app_settings:
            settings_object = json.load(app_settings)
        
        return LulaSafeOptions(secrets_object["ClientId"], secrets_object["ClientSecret"], settings_object["BaseUrl"])
    
    @staticmethod
    def read_from_default_config():
        return LulaSafeOptions.read_from_config("../appsecrets.json", "../appsettings.json")

lula_safe_config = LulaSafeOptions.read_from_default_config()
print(lula_safe_config)

In [ ]:
# Lula PATHs
init_session_path = "v1/login/initialize"
submit_login_path = "v1/login/submit"

### 2. Initiate session

In [ ]:
import requests as req

response = None
try:
    response = req.get(url=f"{lula_safe_config.base_url}/{init_session_path}")
    response = response.json()
except req.exceptions.JSONDecodeError as jde:
    print(f"Failed to parse response into a valid JSON object because of:{os.linesep}", jde)
except req.exceptions.RequestException as e:
    print(f"Failed to obtain a flow id because of:{os.linesep}", e)

flow_id = None
try:
    flow_id = response["id"]
    print(f"FlowId '{flow_id}'")
except KeyError: 
    print("Failed to get an id from JSON, because it is not in the response")
    pp(response)

### 3. Get session token used as bearer

In [ ]:
lula_safe_config = LulaSafeOptions.read_from_default_config()

request_data = {
    "method": "password",
    "password_identifier": lula_safe_config.client_id,
    "password": lula_safe_config.client_secret
}
full_url = f"{lula_safe_config.base_url}/{submit_login_path}?flow={flow_id}"

json_request = json.dumps(request_data).encode('utf-8')

response = None
try:
    response = req.post(url=full_url, data=json_request, headers= {'Content-Type': 'application/json; charset=utf-8'})
    response = response.json()
except req.exceptions.JSONDecodeError as jde:
    print(f"Failed to parse response into a valid JSON object because of:{os.linesep}", jde)
except req.exceptions.RequestException as e:
    print(f"Failed to obtain a session token because of:{os.linesep}", e)

bearer_token = None
try:
    bearer_token = response["session_token"]
    print(f"Bearer Token '{bearer_token}'")
except KeyError:
    print("Failed to get a session_token from JSON, because it is not in the response")
    pp(response)

## Session concept
As long as API must also be usable from client side application (i.e. from browser) first you establish a short leaved session from a back-end. Then you can pass it to front-end and do not worry about it's disclosure. Or use it from back-end too.
So after you got a session Id, use it for later calls.

### Establishing a session
Use bearer token as usual in `Authorization` header to establish a session

In [ ]:
lula_safe_config = LulaSafeOptions.read_from_default_config()

session_id = None

lula_session_endpoint = f"{lula_safe_config.base_url}/risk/v0.1-beta1/sessions"

response = None
try:
    response = req.post(url=lula_session_endpoint, headers={'Authorization': f"Bearer {bearer_token}"})
    response = response.json()
except req.exceptions.JSONDecodeError as jde:
    print(f"Failed to parse response into a valid JSON object because of:{os.linesep}", jde)
except req.exceptions.RequestException as e:
    print(f"Failed to obtain a session_id token because of:{os.linesep}", e)

session_id = None
try:
    session_id = response["sessionId"]
    print(f"SessionId '{session_id}'")
except KeyError:
    print("Failed to get a session_id from JSON, because it is not in the response")
    pp(response)

> **Note**
>
> Having a `sessionId` you no longer need to pass a bearer token. Hence you can pass it to front-end or mobile app

## GraphQL client usage
Let's define a function to read an operation from a `gql` file.

In [ ]:
import asyncio
import os
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

lula_safe_config = LulaSafeOptions.read_from_default_config()

lulasafe_graphql_url = f"{lula_safe_config.base_url}/risk/graphql"
transport = AIOHTTPTransport(lulasafe_graphql_url, headers= {'session-id': session_id})

def read_operation(name: str):
    operations_path = os.path.abspath("../graphql")
    try:
        gql_file = open(f"{operations_path}/{name}", "r")
        data = gql_file.read()
        gql_file.close()
        return data
    except OSError as e:
        print(f"Failed to open operation file because of:{os.linesep}", e.strerror)


## This is an example of how to use async GraphQL API.
## We will use that in the next cell

# async def main():
#     async with Client(
#         transport=transport,
#         fetch_schema_from_transport=True,
#         parse_results=True
#     ) as session:
#         assess_mutation = read_operation("assessMutation.txt")
#         ## ...

# await main()


### Driver Assessment
> **Important**
>
>  Store assessment Id on your back-end to later retrieve the result again

Collect driver data and request an assessment for that driver ([CheckInsuranceAndRequestVehicles.gql](../graphql/CheckInsuranceAndRequestVehicles.gql) mutation is used)

In [ ]:
assessee = {
    "firstName": "NOAH",
    "lastName": "RUSSAW",
    "middleName": "",
    "dateOfBirth": "1962-8-1",
    "phone": "",
    "email": ""
}

address_request = {
    "line1": "8340 BUNCHE DR",
    "line2": "",
    "zipCode": "75243",
    "country": "US",
    "city": "DALLAS",
    "state": "TX"
}

driver_assessment_id = None

async def create_driver_assessment():
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
        parse_results=True
    ) as client:
        assess_mutation = gql(read_operation("CheckInsuranceAndRequestVehicles.gql"))
        params = {"assessee": assessee, "address": address_request}     
        response = await client.execute(assess_mutation, variable_values=params)
        try:
            return response["assess"]["id"]
        except KeyError:
            print("Failed to get an assess.id from JSON, because it is absent in the response")
        

driver_assessment_id = await create_driver_assessment()
f"Driver Assessment Id '{driver_assessment_id}'"

### Getting assessment results later
Get any previous assessment results by assessment Id ([RetrieveInsuranceAndVehiclesResult.gql](../graphql/RetrieveInsuranceAndVehiclesResult.gql) query is used)

In [ ]:
async def get_assessment_results():
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
        parse_results=True
    ) as client:
        assessment_query = gql(read_operation("RetrieveInsuranceAndVehiclesResult.gql"))
        params = {"id": driver_assessment_id}
        response = await client.execute(assessment_query, variable_values=params)
        print("Assessment Results:")
        pp(response)

await get_assessment_results()